In [ ]:
import nltk
import math
from nltk.corpus import gutenberg
from nltk.util import ngrams
from collections import defaultdict, Counter
import random
from typing import List, Dict, Tuple

nltk.download('gutenberg')
nltk.download('punkt')

class NGramAutocomplete:
    def __init__(self, n: int):
        #initialize, n for order of ngram to use; ex. 3 for trigram
        self.n = n
        self.ngram_freqs = defaultdict(Counter)
        self.vocab = set()
        self.start_tokens = []
        
    def preprocess(self, text: str) -> List[str]:
        #preprocess text by tokenizing and cleaning.
        #basic cleaning and tokenization
        tokens = nltk.word_tokenize(text.lower())
        tokens = [token for token in tokens if token.isalpha()]  #remove punc
        return tokens
    
    def train(self, texts: List[str]):
        """
        train model on a list of texts, where
        texts: list of training texts
        """
        for text in texts:
            tokens = self.preprocess(text)
            self.vocab.update(tokens)
            
            #sentence boundary markers
            tokens = ['<START>'] * (self.n - 1) + tokens + ['<END>']
            
            #generate and count  n-grams
            for gram in ngrams(tokens, self.n):
                prefix = gram[:-1]
                next_word = gram[-1]
                self.ngram_freqs[prefix][next_word] += 1
                
                #track sentence starters
                if prefix == ('<START>',) * (self.n - 1):
                    self.start_tokens.append(next_word)
    
    def predict(self, prompt: str, num_suggestions: int = 3) -> List[str]:
        """
        generate autocomplete suggestions for a given prompt
        
        prompt: input text to complete
        num_suggestions: number of suggestions to return
            
        returns list of suggested completions?
        """
        tokens = self.preprocess(prompt)
        
        #empty prompt
        if not tokens:
            return random.choices(self.start_tokens, k=num_suggestions)
        
        #get appropriate context window and find nexts words
        prefix = tuple(tokens[-(self.n - 1):])
        suggestions = self.ngram_freqs.get(prefix, Counter())
        
        # lower-order n-grams if no matches
        if not suggestions and len(prefix) > 1:
            for i in range(1, len(prefix)):
                shorter_prefix = prefix[i:]
                suggestions = self.ngram_freqs.get(shorter_prefix, Counter())
                if suggestions:
                    break
        
        #return most common suggestions (or random ones if none r found)
        if suggestions:
            return [word for word, _ in suggestions.most_common(num_suggestions)]
        else:
            return random.choices(list(self.vocab), k=num_suggestions)
        
        #possibly implement probability calculations for top word recommendations?
    
    def generate_sentence(self, max_length: int = 20) -> str:
        #generate a complete sentence

        tokens = []
        prefix = ('<START>',) * (self.n - 1)
        
        for _ in range(max_length):
            next_word = random.choices(
                list(self.ngram_freqs[prefix].keys()),
                weights=list(self.ngram_freqs[prefix].values()),
                k=1
            )[0] if prefix in self.ngram_freqs else '<END>'
            
            if next_word == '<END>':
                break
                
            tokens.append(next_word)
            prefix = tuple((list(prefix) + [next_word])[-(self.n - 1):])
        
        #capitalize & add period
        sentence = ' '.join(tokens).capitalize()
        if not sentence.endswith(('.', '!', '?')):
            sentence += '.'
            
        return sentence

    def get_ngram_probability(self, context: Tuple[str], word: str, alpha: float = 1.0) -> float:
        #returns the smoothed probability of `word` given `context`.
        
        vocab_size = len(self.vocab) + 1  # for unseen words
        context_counts = self.ngram_freqs.get(context, Counter())
        word_count = context_counts.get(word, 0)
        total_count = sum(context_counts.values())

        # Apply Laplace smoothing
        prob = (word_count + alpha) / (total_count + alpha * vocab_size)
        return prob
    
    def calculate_perplexity(self, test_texts: List[str]) -> float:
        total_log_prob = 0.0
        total_words = 0

        for text in test_texts:
            tokens = self.preprocess(text)
            tokens = ['<START>'] * (self.n - 1) + tokens + ['<END>']

            for i in range(self.n - 1, len(tokens)):
                context = tuple(tokens[i - self.n + 1:i])
                word = tokens[i]

                prob = self.get_ngram_probability(context, word)
                total_log_prob += math.log(prob)
                total_words += 1

        return math.exp(-total_log_prob / total_words) if total_words > 0 else float('inf')
    
#initialize and train
autocomplete = NGramAutocomplete(3)

#im using the ebooks included in nltk so i dont have to download one billion ebooks while testing
gutenberg_files = gutenberg.fileids()
gutenberg_texts = [gutenberg.raw(file_id) for file_id in gutenberg_files]

#train only on jane austen's emma
emma = gutenberg.raw("austen-emma.txt")
autocomplete.train([emma])

print("Suggestions:", autocomplete.predict("she was not"))
print("Generated:", autocomplete.generate_sentence())
print("Perplexity:", autocomplete.calculate_perplexity([emma]))

#uncomment following to train on all gutenberg sample texts, takes a couple minutes to run
"""autocomplete.train(gutenberg_texts)


print("Suggestions:", autocomplete.predict("she was not"))
print("Generated:", autocomplete.generate_sentence())

#perplexity (using held-out test text)
print("Perplexity:", autocomplete.calculate_perplexity(gutenberg_texts))"""



[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\15013\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\15013\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Suggestions: ['to', 'the', 'so']
Generated: Emma by hand and entered on and say every body returning into their troubles with ready sympathy how you were.
Perplexity: 2662.989778834968


'autocomplete.train(gutenberg_texts)\n\n\nprint("Suggestions:", autocomplete.predict("she was not"))\nprint("Generated:", autocomplete.generate_sentence())\n\n#perplexity (using held-out test text)\nprint("Perplexity:", autocomplete.calculate_perplexity(gutenberg_texts))'